In [1]:
import pandas as pd
import re
import string
import collections
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, SnowballStemmer

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
df_train = pd.read_csv('/content/gdrive/MyDrive/7mo semestre/Deep Learning/train_mod.csv')

In [4]:
df_train=df_train[['label','text']]

In [5]:
df_train = df_train.dropna(subset=['text'])
import numpy as np
df_train['label'].astype(np.int64)
df_train['text'].astype(str)

0         despite fact played small portion game music h...
1         bought charger jul worked ok design nice conve...
2         check maha energys website powerex mhcf charge...
3         reviewed quite bit combo players hesitant due ...
4         also began incorrect disc problems ive read vc...
                                ...                        
399994    bought thomas son huge thomas fan huge set roo...
399995    son recieved birthday gift months ago loved ev...
399996    bought toy son loves thomas toys need one batt...
399997    compilation wide range mitfords articles best ...
399998    dvd disappointment get hoping see substantial ...
Name: text, Length: 399998, dtype: object

In [6]:
#dividimos el data set de train en train y validation
size=int(len(df_train)*0.7)
train_df=df_train[:size]
val_df=df_train[size:]

In [7]:
print(train_df['label'].value_counts())
print(val_df['label'].value_counts())

1    1262962
0    1257030
Name: label, dtype: int64
0    542967
1    537030
Name: label, dtype: int64


# Crear diccionario

In [15]:
# Maximum number of words to be considered in the vocabulary
max_words = 10000
# Maximum number of tokens in a sequence
max_len = 200
# Tokenizer
tokenizer = Tokenizer(num_words = max_words)
# Snap tokenizer to text data
tokenizer.fit_on_texts(train_df['text'])
# Converts texts into strings of numbers
sequences_train = tokenizer.texts_to_sequences(train_df['text'])
sequences_val = tokenizer.texts_to_sequences(val_df['text'])
# Mapping words to indexes
word_index = tokenizer.word_index

In [19]:
import json

# Guardar el diccionario en un archivo
with open('word_index.json', 'w') as f:
    json.dump(word_index, f)

#Importar diccionario para ajustar las secuencias

In [8]:
import json

# Cargar el vocabulario desde el archivo guardado (por ejemplo, word_index.json)
with open('/content/gdrive/MyDrive/7mo semestre/Deep Learning/word_index.json', 'r') as f:
    loaded_word_index = json.load(f)

In [9]:
from keras.preprocessing.text import Tokenizer

# Crear un nuevo Tokenizer
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)

# Inicializar el Tokenizer con el vocabulario cargado
tokenizer.word_index = loaded_word_index

In [10]:
# Convertir el texto a secuncia de  enteros
sequences_train = tokenizer.texts_to_sequences(train_df['text'])
sequences_val = tokenizer.texts_to_sequences(val_df['text'])

In [11]:

max_len = 200
data_train = pad_sequences(sequences_train, maxlen = max_len)
data_val = pad_sequences(sequences_val, maxlen = max_len)

In [12]:
# Crear el modelo
model = tf.keras.Sequential()
model.add(Embedding(max_words, 16, input_length = max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation = 'sigmoid'))

# Compilar el moedlo
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160017 (625.07 KB)
Trainable params: 160017 (625.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
#epochs 15
#steps_per_epoch=25
history = model.fit(data_train, train_df['label'], epochs = 15,steps_per_epoch=50, batch_size = 64, validation_data = (data_val, val_df['label']))

Epoch 1/15
50/50 [==============================] - 43s 867ms/step - loss: 0.6926 - accuracy: 0.5466 - val_loss: 0.6918 - val_accuracy: 0.5035
Epoch 2/15
50/50 [==============================] - 33s 683ms/step - loss: 0.6912 - accuracy: 0.6225 - val_loss: 0.6902 - val_accuracy: 0.5483
Epoch 3/15
50/50 [==============================] - 41s 842ms/step - loss: 0.6892 - accuracy: 0.5978 - val_loss: 0.6884 - val_accuracy: 0.5156
Epoch 4/15
50/50 [==============================] - 41s 846ms/step - loss: 0.6876 - accuracy: 0.6584 - val_loss: 0.6856 - val_accuracy: 0.8119
Epoch 5/15
50/50 [==============================] - 35s 710ms/step - loss: 0.6839 - accuracy: 0.7828 - val_loss: 0.6821 - val_accuracy: 0.7650
Epoch 6/15
50/50 [==============================] - 41s 842ms/step - loss: 0.6803 - accuracy: 0.7344 - val_loss: 0.6779 - val_accuracy: 0.7033
Epoch 7/15
50/50 [==============================] - 33s 672ms/step - loss: 0.6753 - accuracy: 0.7725 - val_loss: 0.6729 - val_accuracy: 0.7014

In [14]:
loss, accuracy = model.evaluate(data_val, val_df['label'], verbose = 0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 82.548380


In [15]:
# Guardar la arquitectura y los pesos del modelo en un archivo
model.save('/content/gdrive/MyDrive/7mo semestre/Deep Learning/modelo_entrenado.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


"# También puedes guardar el Tokenizer si es necesario para la tokenización\nwith open('tokenizer.pickle', 'wb') as handle:\n    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)"